In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/assignment-3/data/Task1/dev.csv
/kaggle/input/assignment-3/data/Task1/sample_test.csv
/kaggle/input/assignment-3/data/Task1/sample_demo.csv
/kaggle/input/assignment-3/data/Task1/train.csv
/kaggle/input/assignment-3/data/Task2/train_data.json
/kaggle/input/assignment-3/data/Task2/validation_data.json
/kaggle/input/assignment-3/data/Task2/test.json
/kaggle/input/assignment-3/data/A3_task1_data_files/dev.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/sample_test.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/sample_demo.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/train.csv
/kaggle/input/assignment-3/A3_task1_data_files/dev.csv
/kaggle/input/assignment-3/A3_task1_data_files/sample_test.csv
/kaggle/input/assignment-3/A3_task1_data_files/sample_demo.csv
/kaggle/input/assignment-3/A3_task1_data_files/train.csv
/kaggle/input/assignment-3/assignment-3/dev.csv
/kaggle/input/assignment-3/assignment-3/sample_test.csv
/kaggle/input/assignment-3/assignment

In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.1 MB/s eta 0:00:00


In [3]:
import os
import csv
import math
import torch
import logging
import pandas as pd
from datetime import datetime
from torch.utils.data import DataLoader, Dataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample

from sentence_transformers import SentenceTransformer,util
import torch
import scipy
from scipy.stats import pearsonr

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f'Using GPU: {torch.cuda.get_device_name()}')
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    logging.info('Using CPU')
    print('Using CPU')

Using GPU: Tesla P100-PCIE-16GB


In [4]:
path = "/kaggle/input/assignment-3/data/Task1/sample_test.csv"

In [5]:
df = pd.read_csv(path,sep="\t")
if "setence1" in df.columns:
    df.rename(columns={"setence1":"sentence1"},inplace=True)
else:
    pass
df

,id,sentence1,sentence2
0,1,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,2,A young child is riding a horse.,A child is riding a horse.
2,3,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,4,A woman is playing the guitar.,A man is playing guitar.
4,5,A woman is playing the flute.,A man is playing a flute.
5,6,A woman is cutting an onion.,A man is cutting onions.


In [6]:
model = torch.load("/kaggle/input/a3-1c-models/all-MiniLM-L6-v2-final.pth")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [7]:
sentence_set_1 = df["sentence1"].to_numpy()
sentence_set_2 = df["sentence2"].to_numpy()

In [8]:
encode_sentence_set_1 = model.encode(sentence_set_1)
encode_sentence_set_2 = model.encode(sentence_set_2)
cosine_similarities = util.cos_sim(encode_sentence_set_1,encode_sentence_set_2).tolist()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
predicted_scores=[]
for i in range(len(cosine_similarities)):
    cos_score = (cosine_similarities[i][i]+1)/2
    predicted_scores.append(cos_score*5)
    

    
if 'id' in df.columns:
    data={
        'id':df["id"],
       'sentence1':sentence_set_1,
        'sentence2':sentence_set_2,
        'predicted_score':predicted_scores
    }
else:
    data={
        'sentence1':sentence_set_1,
        'sentence2':sentence_set_2,
        'predicted_score':predicted_scores
   }
final_dataframe = pd.DataFrame(data)
final_dataframe.to_csv("sample_demo.csv")
final_dataframe

,id,sentence1,sentence2,predicted_score
0,1,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,4.986553
1,2,A young child is riding a horse.,A child is riding a horse.,4.944526
2,3,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,4.978417
3,4,A woman is playing the guitar.,A man is playing guitar.,4.019412
4,5,A woman is playing the flute.,A man is playing a flute.,4.512961
5,6,A woman is cutting an onion.,A man is cutting onions.,4.416580
